In [ ]:
import numpy as np
import pandas as pd

import core.artificial_signal_generators as sig_gen
import core.finance as fin
import core.statistics as stats
import helpers.unit_test as hut

In [ ]:
def _generate_series(seed) -> pd.Series:
    arma_process = sig_gen.ArmaProcess([], [])
    realization = arma_process.generate_sample(
        {"start": "2000-01-01", "periods": 75, "freq": "D"},
        scale=1,
        seed=seed,
    )
    return realization

Generating series with `D` daily frequency and 75 observations.

In [ ]:
srs = _generate_series(seed=1)
srs

## Testing current implementation

Current verion of `fin.compute_average_holding_period()` is ugly but working as it is supposed to:

In [ ]:
result_in_days = fin.compute_average_holding_period(srs)
result_in_days

Default result if appled to srs is 0.727.

Checking 'by-hand'

In [ ]:
srs.abs().mean() / srs.diff().abs().mean()

Seems correct.

Therefore, if we want to output the calculated average holding period in months, 
the output should be equal to `0.72768/30`.

PLEASE, note, if the statement above is incorrect and results for months should be different in this case - tell me and do not read everything below, brcause this assumption is crucail in my thinking here.

In [ ]:
result_in_month = fin.compute_average_holding_period(srs, unit='M')
result_in_month

In [ ]:
result_in_days/30

So we see that the function is working correctly.

## Approach with `resample()` is not really applicable

First, let's take a look at th suggested function in https://github.com/alphamatic/amp/pull/378#issuecomment-647843776

In [ ]:
def get_mean_holding(srs: pd.Series, freq: str) -> float:
    num = srs.abs().resample(freq).sum().mean()
    denom = srs.diff().abs().resample(freq).sum().mean()
    return num / denom

The problem with it is that `resample()` is applied to both `num` and `denum` which excludes any point in the resampling.
Because in both `srs.abs()` and `srs.diff().abs()` we calculate the mean by summarizing all of these series values and 
dividing them by their size. in other words, `srs.abs().mean() = srs.abs().sum()/ srs.abs().size` and 
`srs.diff().abs().mean() = srs.diff().abs().sum()/ srs.diff().abs().size`.

When we apply `.resample(freq).sum()`, we do not change the sum of the values, because `srs.abs().sum() = srs.abs().resample(freq).sum().sum()` and  `srs.diff().abs().sum() = srs.diff().abs().resample(freq).sum().sum()`.

Therefore, resampling impacts only the size of the series, that has to become smaller. However, if we apply resampling to the both `num` and `denum`, then both their sizes reduce by the same rate and are still equal. So when we do `num/denum` this sizes eradicate themselves and in the end we have the same output for all the frequencies which in my opinion doesn't make any sense:

In [ ]:
get_mean_holding(srs, freq='D')

In [ ]:
get_mean_holding(srs, freq='M')

In [ ]:
get_mean_holding(srs, freq='Y')

You can see that the results are almost equal, which is not what we want.

I've tried to resolve this problem by resampling only `denum`, naively thinking that it will resolve everything.

In [ ]:
def get_mean_holding_1(srs: pd.Series, freq: str) -> float:
    num = srs.abs().mean()
    denom = srs.diff().abs().resample(freq).sum().mean()
    return num / denom

In [ ]:
get_mean_holding_1(srs, freq='D')

In [ ]:
get_mean_holding_1(srs, freq='M')

Looks fine at the first sight. However, if we check with the correct result, there is a difference:

In [ ]:
get_mean_holding_1(srs, freq='D')/30 

In [ ]:
(get_mean_holding_1(srs, freq='M')) / (get_mean_holding_1(srs, freq='D')/30) 

20%% higher than it should be! It is more sensible with years:

In [ ]:
get_mean_holding_1(srs, freq='Y')

In [ ]:
get_mean_holding_1(srs, freq='D')/365

In [ ]:
(get_mean_holding_1(srs, freq='Y')) / (get_mean_holding_1(srs, freq='D')/365)

Almost 5 times higher. It happens because after resampling it calculates size wrongly.
For example, `srs.diff().abs().size` = 75 so the size of resampled series should be 75/30 = 2.5 in order to compute the mean
correclty. However:

In [ ]:
srs.diff().abs().size

In [ ]:
srs.diff().abs().resample('M').sum().size

In [ ]:
srs.diff().abs().resample('M').sum()

As we see it counts March as a full month and this biases our results.
The same happens for all the other cases and, therefore, the approach is not really applicable.

This means that we should compute the average holding period with the input frequency and then multiply it by the frequency coefficient which is more or less fixed for all the frequency pairs (`D`/`M` = 1/30, `W`/`Y`=52 etc.)

The same logic is implemented in the current version.

Julia has offered to use the following format to do this:

In [ ]:
pd.date_range("2020-01-01", freq='D', periods=1)[-1]

However, I believe that this approach is sort of biased too.

It does not output the date that is after the number of 1 `freq` datetime, but has it's own indexing and just chooses the date at the end of specified `freq`.
For example:

In [ ]:
pd.date_range("2020-01-01", freq="W", periods=1)[-1]

In [ ]:
pd.date_range("2020-01-04", freq="W", periods=1)[-1]

By setting `freq='W'` we did not obtain a date in 7 days from specified. Moreover, for different dates in hte input the output is the same. This can bias our calculations and it is much more visible on the other examples:

Same output for `freq='M` and 2 dates in the specified month: 

In [ ]:
pd.date_range("2020-01-07", freq="M", periods=1)[-1]

In [ ]:
pd.date_range("2020-01-18", freq="M", periods=1)[-1]

Same output for `freq='Y` and 2 dates in the specified year: 

In [ ]:
pd.date_range("2020-03-07", freq="Y", periods=1)[-1]

In [ ]:
pd.date_range("2020-09-19", freq="Y", periods=1)[-1]

Therefore, both approaches do not satisfy our need.

Here is an alternative: to make a function that will return the ratio of one frequency lenght in time to another.
Simplified example:

In [ ]:
def get_freq_coef(freq_1: str, freq_2: str) -> float:
    time_dict = {
        'D': {'D': 1, 'M': 30, 'Y': 365},
        'M': {'D': 1/30, 'M': 1, 'Y':12},
        'Y': {'D': 1/365, 'M':1/12, 'Y':1}
                }
    return time_dict[freq_1][freq_2]

In [ ]:
get_freq_coef(freq_1='Y', freq_2='D')

In [ ]:
get_freq_coef(freq_1='D', freq_2='Y')

In [ ]:
get_freq_coef(freq_1='M', freq_2='Y')

In [ ]:
get_freq_coef(freq_1='Y', freq_2='M')

Something like this can be created for all the relevant time frequencies and then be used in other cases when we need to convert an amount of days to an amount of month etc.